# Decidi il tuo domani: supporto alla scelta universitaria

L'obiettivo è creare un sistema per consigliare facoltà compatibili con gli interessi di una persona. Sebbene esistano già sistemi con questo scopo, il loro utilizzo non è molto user-friendly, poiché richiedono la compilazione di un numero eccessivo di domande.

Il sistema proposto ridurrà il numero di domande da compilare e permetterà all'utente di descrivere liberamente i propri interessi attraverso un testo scritto.

## Struttura del progetto

![./img/schema.png](./img/schema.png)

Lo schema presenta due tipi di elementi:

- Rettangolo: rappresenta dati statici (come database) o dati processati da una funzione.
- Freccia: rappresenta una funzione che prende in input dei dati e restituisce in output altri dati.

I numeri accanto ai rettangoli e alle frecce servono unicamente a richiamare gli elementi durante la descrizione delle varie componenti. I numeri di color arancione servono a far riferimento ai blocchi contententi dati, mentre quelli blu per le funzioni.

In [ ]:
# librerie usate
import json
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import math
import mysql.connector
import pandas as pd
from fuzzywuzzy import fuzz
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

## Funzionamento del sistema

Il sistema si aspetta in input i dati nella tabella 1:
- un vettore di numeri che rappresenta la profilazione dell'utente (riquadro giallo)
- un testo con gli interessi dell'utente (riquadro verde acqua)

Il vettore di numeri viene passato alla Funzione 1, che, sfruttando le informazioni sugli utenti profilati (Tabella 2), individua gli utenti simili a quello a cui vogliamo consigliare le facoltà. In questo modo, otteniamo una prima idea delle facoltà che potrebbero piacere all'utente (strategia di collaborative filtering). La tabella in output associa ad ogni facoltà una score.

Il testo con gli interessi dell'utente viene passato alla Funzione 2, che divide le parole della frase in due gruppi: parole con sentiment positivo e parole con sentiment negativo (Tabella 4). Ad esempio, se l'utente digita "Mi piace la matematica ma non la storia", ci aspettiamo che la parola "matematica" venga classificata come parola con sentiment positivo, mentre "storia" con sentiment negativo.

Successivamente, la Funzione 4 prende in input le parole suddivise per sentiment (Tabella 4) e partiziona le facoltà del database in quelle che potrebbero interessare all'utente (punteggio positivo) e quelle che potrebbero non piacere (punteggio negativo) e da in output la tabella 6.

In fine la funzione 5 sfrutta le informazioni trovate finora per dare in output le facoltà suggerite all'utente. 

Questa strategia ibrida sfrutta la filosofia della serie TV "Quattro ristoranti": i consigli della tabella 3 possono essere confermati o ribaltati da quelli della tabella 6.

## Descrizione dettagliata delle funzioni

#### Funzione 1
- **Input:** Vettore di numeri (chiamato anche query, riquadro giallo in Tabella 1), Utenti profilati (Tabella 2)
- **Output:** Vettore di facoltà con associato uno score (Tabella 3)

Gli utenti sono profilati tramite vettori di lunghezza fissa. Si sfrutta la distanza del coseno tra la query e ogni utente profilato. In output viene fornita una lista delle facoltà consigliate, con uno score associato a ciascuna facoltà: lo score è maggiore se la distanza del coseno è minore.

#### Funzione 2
- **Input:** Stringa con gli interessi dell'utente (riquadro verde acqua in Tabella 1)
- **Output:** Vettore di parole associate a sentiment positivo (riquadro verde in Tabella 4), Vettore di parole associate a sentiment negativo (riquadro rosso in Tabella 4)

La Funzione 2 analizza il testo con gli interessi dell'utente nel seguente modo:
1. Spezza la frase in sotto-frasi.
2. Per ogni sotto-frase:
   - Estrae una lista di tutte le parole che sono nomi o verbi.
   - Applica VADER (Valence Aware Dictionary and sEntiment Reasoner) per determinare il sentiment della sotto-frase.
   - Se il sentiment è positivo, aggiunge le parole al vettore delle parole positive.
   - Se il sentiment è negativo, aggiunge le parole al vettore delle parole negative.
   - Se il sentiment è neutro, si basa sulla sotto-frase precedente:
     - Se il sentiment della sotto-frase precedente era positivo (negativo) e non ci sono negazioni nella sotto-frase corrente, aggiunge le parole al vettore delle parole positive (negative).
     - Se c'è una negazione nella sotto-frase corrente, aggiunge le parole al vettore delle parole con sentiment opposto (parole negative se il sentiment precedente era positivo, e viceversa).

Per negazioni si intendono parole come: 'non', 'ma', 'però'. Queste parole, nella pratica, spesso portano a un cambio di polarità nella frase successiva, influenzando il sentiment complessivo della frase.

#### Funzione 3
Descritta successivamente perché viene eseguita solo una volta in maniera offline per la creazione del database con le informazioni delle facoltà

#### Funzione 4
- **Input:** Vettore di parole associate a sentiment positivo (riquadro verde in Tabella 4), Vettore di parole associate a sentiment negativo (riquadro rosso in Tabella 4), Database con le informazioni delle facoltà
- **Output:** Vettore di facoltà con associato uno score (Tabella 6)

La Funzione 4 esegue il seguente algoritmo sia per le parole associate a sentiment positivo che negativo:

1. Mantengo le parole che compaiono nelle descrizioni delle facoltà (k parole)
2. Creo una tabella con k colonne e con numero di righe pari alle facoltà. Per ogni facoltà, associo alla i-esima colonna la TF della i-esima parola nella descrizione della facoltà moltiplicata per la sua IDF
3. Creo un vettore di k elementi associato al vettore di k parole. L'i-esimo elemento è il prodotto della TF della parola nel vettore di parole moltiplicato per la sua IDF.
4. Utilizzo la distanza del coseno per calcolare la similarità tra il vettore calcolato al punto 3 e le righe della tabella calcolata al punto 3. A ciascuna facoltà viene assegnato uno score: minore è la distanza del coseno maggiore è lo score.
5. Se le parole erano associate a sentiment negativo, moltiplico per -1 gli score delle facoltà pertinenti. In questo modo, vengono identificate le facoltà "vicine" agli interessi dell'utente ed associo un punteggio positivo, mentre quelle "vicine" a ciò che non piace all'utente vengono considerate con uno score negativo.

#### Funzione 5
- **Input:** Vettore di facoltà con associato uno score (Tabella 3), Vettore di facoltà con associato uno score (Tabella 6)
- **Output:** Vettore di facoltà con associato uno score (Tabella 7)

Per ogni facoltà sotto gli score ottenuti nella tabella 3 e 7. In questo modo se una facoltà ha ottenuto un punteggio positivo in entrambe le tabella è probabile che venga consigliata. Se il punteggio è negativo invece il punteggio nella tabella 3 viene abbassato. Questo processo di combinazione dei punteggi fornisce una valutazione più accurata delle facoltà da consigliare all'utente, tenendo conto sia delle preferenze degli utenti simili sia degli interessi dell'utente.

## Raccolta dati

Il sistema per funzionare richiede due fonti di dati:

- Dati sugli utenti già iscritti a una facoltà (Tabella 2)
- Dati sulle facoltà (Tabella 5)


##### Estrazione dati sugli utenti
La creazione della Tabella 2 avviene tramite un form fornito esclusivamente agli studenti universitari. Il form è composto da domande a scelta multipla: nella prima viene chiesto di inserire la facoltà frequentata e successivamente vengono proposte domande per profilare gli interessi dell'utente.

##### Estrazione dati sulle facoltà
La creazione della Tabella 5 sfrutta le informazioni presenti sul sito dell'Università di Catania. Per ciascuna facoltà dell'ateneo sono state estratte la descrizione del corso, gli obiettivi formativi e gli sbocchi professionali.

La sfida principale nel fare web scraping sul sito dell'università è la scarsa strutturazione dei siti dei vari dipartimenti: alcuni presentano le informazioni richieste nella homepage, mentre altri utilizzano menù a tendina che reindirizzano a pagine specifiche. Un altro problema è l'assenza di classi nel codice HTML, rendendo l'estrazione automatizzata più complessa.

Inizialmente, avrei voluto fare web scraping per accelerare il processo, ma i problemi descritti lo hanno reso impraticabile. Pertanto, ho estratto manualmente le informazioni richieste da ogni dipartimento e ho creato un array di oggetti JSON. Le informazioni sulle facoltà sono state tradotte in inglese perché le librerie per l'analisi del testo sono maggiormente supportate per questa lingua.

![JSON_info_faculties.png](./img/JSON_info_faculties.png)

In [ ]:
# FUNZIONE PER IL PRE-PROCESSING DEL TESTO

lemmatizer = WordNetLemmatizer()

def clean_text(text: str) -> str:
    # lowercase the text
    text = text.lower()
    # tokenize the text
    words = word_tokenize(text)
    # remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]
    # remove special characters
    words = [word for word in words if word.isalnum()]
    # lemmatize the words
    words = [lemmatizer.lemmatize(word) for word in words]
    # keep only nouns
    words = [word for word in words if wordnet.synsets(word) and wordnet.synsets(word)[0].pos() == 'n']

    # join the words
    return ' '.join(words)

## Creazione dei database

I dati raccolti devono essere immagazzinati in due database: per la profilazione dell'utente e per le informazioni delle facoltà.

#### Creazione database profilazione utenti

Per creare il database per immagazzinare le informazioni raccolte tramite il form google, partiamo dall'analizzare il file .csv con le informazioni. Il form è reperibile al seguente [link](https://docs.google.com/forms/d/e/1FAIpQLScXo5_wbRJN6U-SsDn9PMd4hd6B3IFJhNrwZ5scvbOaQ47EWA/viewform?usp=sf_link).

In [ ]:
df = pd.read_csv('./data/user_profilation.csv')

# show all columns
print(df.columns)

In [ ]:
# pie chart
def plot_pie_chart(df, column_name):
    # Get the value counts for the column
    counts = df[column_name].value_counts()
    
    # Plot the pie chart
    plt.figure(figsize=(8, 8))
    plt.pie(counts, labels=counts.index, autopct='%1.1f%%', startangle=140)
    plt.title(f'Pie Chart for {column_name}')
    plt.legend(counts.index, title=column_name, loc="best", bbox_to_anchor=(1, 0, 0.5, 1))
    plt.tight_layout()
    plt.show()
    
def plot_histogram(df, column_name):
    plt.figure(figsize=(10, 6))  # opzionale: imposta le dimensioni della figura
    df[column_name].hist(bins=len(df[column_name].unique()), edgecolor='black')  # bins imposta il numero di bin nell'istogramma
    plt.title(f'Histogram for {column_name}')
    plt.xlabel(column_name)
    plt.ylabel('Frequency')
    plt.xticks(rotation=90)

# Mostra l'istogramma
plt.show()



In [ ]:
plot_pie_chart(df, 'Tipologia:')
plot_pie_chart(df, 'Sesso')
plot_histogram(df, 'Età')
plot_pie_chart(df, 'Preferisco studiare materie:')
plot_pie_chart(df, 'Studio perché voglio aiutare le persone dal punto di vista: (ciò che preferisci di più)')
plot_pie_chart(df, 'Mi soddisfa più: (ciò che preferisci di più)')
plot_pie_chart(df, 'Quando studio, mi piace avere più risposte per la stessa domanda:')
plot_histogram(df, 'Fra questi obiettivi mi rispecchio più in: (ciò che preferisci di più)')
plot_histogram(df, 'In quale ambiente di lavoro ti senti più a tuo agio?')
plot_histogram(df, 'Quanto sei soddisfatto della tua scelta accademica?')

Non tutte le colonne sono necessarie per la creazione del db.

In [ ]:
columns_to_remove = ['Informazioni cronologiche', "Anno accademico (Per la magistrale parti dal primo anno)", "Tipologia:", "Sesso", "Età", "Pareri sulla tua scelta o miglioramenti da apportare al form."]
df = df.drop(columns=columns_to_remove)

Associamo ad ogni colonna un intero, così da semplificare i nomi.

In [ ]:
# associate to each column a unique integer to simplify the column names
column_names = df.columns
column_map_original_names = {
    key: value for key, value in enumerate(column_names)
}

# rename each column with the associated integer
new_columns = {old_col: i for i, old_col in enumerate(df.columns)}
new_columns["Quale corso frequenti? "] = 'faculty'
new_columns["Quanto sei soddisfatto della tua scelta accademica?"] = 'rate'
df = df.rename(columns=new_columns)

In [ ]:
display(df)

Notiamo che la colonna 0 ha valori non omogenei. Standardizziamo i dati.

In [ ]:
# function that returns the index of the most similar word of an array
def most_similar_word(word, word_array):
    max_similarity = 0
    most_similar_word_index = None

    for i, w in enumerate(word_array):
        # Combining multiple fuzzy matching scores
        ratio = fuzz.ratio(word, w)
        partial_ratio = fuzz.partial_ratio(word, w)
        token_sort_ratio = fuzz.token_sort_ratio(word, w)
        
        # We can weight partial_ratio more to give precedence to matches with consecutive letters
        combined_score = (ratio + 2 * partial_ratio + token_sort_ratio) / 4
        
        if combined_score > max_similarity:
            max_similarity = combined_score
            most_similar_word_index = i

    return most_similar_word_index

In [ ]:
def extract_code(string):
    pattern = r'(l-|lm-|l |lm |l|lm)(\d+)'
    match = re.search(pattern, string)
    if match:
        prefix = match.group(1).strip()
        number = match.group(2)
        return prefix + ("" if '-' in prefix else "-") + number
    else:
        return None
    
columns_new_data = []
with open('./data/faculties.json') as f:
    faculties = json.load(f)['faculties']
    # all valid names
    faculties_names = [faculty['name'].lower() for faculty in faculties]
    # all valid codes
    faculties_codes = [faculty['code'].lower() for faculty in faculties]
    for reply in df['faculty']:
        # read each reply and try to standardize it
        reply = reply.lower()
        # if the length of the reply is short then it probably means that 
        # the user input the code of the faculty
        if len(reply) < 5 :
            code = extract_code(reply)
            if code is not None:
                columns_new_data.append(faculties_names[most_similar_word(code, faculties_codes)])
                continue
        most_similar_word_index = most_similar_word(reply, faculties_names)
        columns_new_data.append(faculties_names[most_similar_word_index])
                                                        
df['faculty'] = columns_new_data
display(df)

Il prossimo step è quello di trasformare i dati categoriali in numerici. Di seguito è riportato un mapping delle risposte da categoriali a numeriche. Il criterio usato è stato quello di assegnare a risposte più umanistiche valori negativi e scientifiche valori positivi.

In [ ]:
mapping = {
    1: {
        "Artistico": -4,
        "Musicale": -3,
        "Classico": -2,
        "Linguistico": -1,
        "Scienze umane": 0,
        "Economico": 1,
        "Tecnologico": 2,
        "Scientifico": 3,
        "Professionale": 4
    },
    2: {
        "Scientifiche": 1,
        "Umanistiche": -1
    },
    3: {
        "Pratica": 0,
        "Teoria": 1
    },
    4: {
        "artistico (es. Stilista, Designer, ...)": -5,
        "culinario (es. Cuoco, ...)": -4,
        "turistico (es. Guida Turistica, ...)": -3,
        "sociale (es. Assistente Sociale, ...)": -2,
        "educativo (es. Insegnante, ...)": -1,
        "civile (es. Avvocato, Politico, ...)": 0,
        "ambientale (es. Agricoltore, Attivista, ...)": 1,
        "economico (es. Impiegato in banca, Economista, ...)": 2,
        "tecnologico (es. Sviluppatore, Progettista, ...)": 3,
        "sanitario (es. Dottore, Infermiere, Psicologo, ...)": 4,
        "della ricerca (es. Fisico, Matematico, Storico, ...)": 5
    },
    5: {
        "creare": -3,
        "comunicare": -2,
        "progettare": -1,
        "comprendere": 0,
        "analizzare": 1,
        "ricercare": 2,
        "aiutare": 3
    },
    6: {
        "No": -1,
        "Sì": 1
    },
    7: {
        "espandere le mie conoscenze e competenze": -3,
        "fare carriera": -2,
        "esplorare nuove culture": -1,
        "esprimere la mia creatività ed originalità": 0,
        "contribuire al benessere degli altri": 1,
        "fare una differenza nel mondo": 2,
        "promuovere la giustizia": 3,
        "promuovere l'innovazione ed il cambiamento": 4
    },
    8: {
        "Un ambiente creativo e innovativo": -3,
        "Un ambiente dinamico e stimolante": -2,
        "Un ambiente collaborativo e team-oriented": -1,
        "Un ambiente multiculturale e diversificato": 0,
        "Un ambiente autonomo e indipendente": 1,
        "Un ambiente strutturato e ben organizzato": 2
    }
}


In [ ]:

def convert_categorical_to_numeric(df, mapping):
    # Itera attraverso ciascuna domanda nel mapping
    for question, map_dict in mapping.items():
        
        if question in df.columns:
            # Converti in valore numerico
            df[question] = df[question].map(map_dict)
    return df

convert_categorical_to_numeric(df, mapping)

display(df)


In [ ]:
# funzione per calcolare la similarità del coseno fra due righe di un dataframe
def calculate_cosine_distance(df, x, y):
    # Seleziona le colonne numeriche create dal mapping
    numeric_columns = [i for i in range(1, 9)]
    
    # Estrai le due righe specificate
    vector_x = df.loc[x, numeric_columns].values
    vector_y = df.loc[y, numeric_columns].values
    
    cos_sim = cosine_similarity([vector_x], [vector_y])
    
    return cos_sim

In [ ]:
# salviamo i dati in un json nel seguente formato

"""
"subjective_data": [
    {
        "faculty": "faculty_1"
        "1": x,
        ...
        "8": z
    },
    {
        "faculty": "faculty_2"
        "1": a,
        ...
        "8": b
    }
]
"""

data_list = {
    "subjective_data": df.to_dict(orient='records')
}

with open("./data/subjective_data.json", 'w') as file:
    json.dump(data_list, file, indent=4)

In [ ]:
plot_histogram(df, "faculty")

Notiamo che la maggior parte delle risposte provengono da studenti che hanno frequentato informatica, però nei sistemi di raccomandazione non sempre il dataset è bilanciato. Un buon sistema dovrebbe funzionare anche con dataset sbilanciati. Una miglioria da apportare sarebbe quella di creare una campagna per spargere il form a tutte le facoltà in modo da coprirle tutte.

#### Creazione database per le facoltà

![DB_faculties.png](./img/DB_faculties_sql.png)

La relazione fra parola e facoltà è di tipo "molti a molti". Per ogni parola viene calcolata la sua IDF e per ogni facoltà viene calcolata la TF di ogni parola che compare nella sua descrizione. Facendo ciò abbiamo pre calcolato le informazioni che sfrutta l'algoritmo. Di seguito è riportato il codice per creare il database.

Il file .sql è contenuto nel file schema.sql (Prima di continuare con il notebook assicurati che il db sia attivo).

![DB_faculties.png](./img/DB_faculties_schema.png)


Prima di riempire il DB con le informazioni richieste creo un JSON con queste caratteristiche:
    
![db_as_json_faculties.png](./img/db_as_json_faculties.png)

In [ ]:
# FUNZIONI DI SUPPORTO PER CALCOLARE IL TD E IDF

def tf(words: list[str]) -> dict[str, float]:
    tf = {}
    for word in words:
        if word in tf:
            tf[word] += 1
        else:
            tf[word] = 1
    total_words = len(words)
    for word in tf:
        tf[word] /= total_words
    return tf


def idf(faculties: list[dict[str, any]]):
    idf = {}
    doc_count = len(faculties)
    all_words = set([word for doc in faculties for word in doc['description'].split()])
    for word in all_words:
        docs_with_word = 0
        for doc in faculties:
            if word in doc['tf']:
                docs_with_word += 1
        idf[word] = 1 + math.log(doc_count / docs_with_word)
    return idf

In [ ]:
db_faculties = {
    'faculties': [],
    'idf': {}
}

with open('./data/faculties_infos.json', 'r') as file:
    faculties = json.load(file)['faculties']
    for faculty in faculties:
        f = {}
        f['code'] = faculty['code']
        f['name'] = faculty['name']
        f['description'] = clean_text(faculty['description']) + clean_text(faculty['goals']) + clean_text(faculty['jobs'])
        f['tf'] = tf(f['description'].split())
        db_faculties['faculties'].append(f.copy())

    db_faculties['idf'] = idf(db_faculties['faculties'])

Ora che ho creato l'oggetto db con tutte le informazioni su TF e IDF, posso salvare tutto sul database.

In [ ]:
# CONNESSIONE AL DATABASE

db = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='faculties_recommender'
)

if db.is_connected():
    print('Connected to the MySQL database')
else:
    # shut down the program if the connection to the database is not established
    print('Connection to the MySQL database failed')
    exit()

In [ ]:
cursor = db.cursor(buffered=True)

# INSERISCO DENTRO LA TABELLA "faculties" TUTTE LE FACOLTA'
for faculty in db_faculties['faculties']:
    cursor.execute("INSERT INTO faculties (code, name) VALUES (%s, %s)", (faculty['code'], faculty['name']))
    db.commit()
print("Tabella faculties riempita")

# INSERISCO DENTRO LA TABELLA "keywords" TUTTE LE PAROLE A DISPOSIZIONE
idf = db_faculties['idf']
for keyword in idf.keys():
    cursor.execute("INSERT INTO keywords (word, idf) VALUES (%s, %s)", (keyword, idf[keyword]))
    db.commit()
print("Tabella keywords riempita")

# INSERISCO DENTRO LA TABELLA "faculties_keywords" L'ASSOCIAZIONE FRA OGNI PAROLA E LA FACOLTA' DOVE ESSA COMPARE
for faculty in db_faculties['faculties']:
    # retrieve faculty ID
    cursor.execute("SELECT id FROM faculties WHERE code = %s", (faculty['code'],))
    faculty_id = cursor.fetchone()[0]

    for word in faculty['tf'].keys():
        # retrieve keyword ID
        cursor.execute("SELECT id FROM keywords WHERE word = %s", (word,))
        keyword_id = cursor.fetchone()[0]
        
        insert_query = "INSERT INTO faculties_keywords (faculty_id, keyword_id, tf) VALUES (%s, %s, %s)"
        insert_data = (faculty_id, keyword_id, faculty['tf'][word])
        cursor.execute(insert_query, insert_data)
db.commit()
print("Tabella faculties_keywords riempita")
cursor.close()

In questo modo abbiamo creato il database per le informazioni in merito alle facoltà.

## Schema UML del sistema di raccomandazione

Finita l'introduzione sul pre-processing dei dati, in questa sezione viene mostrato lo schema UML del codice usato per la creazione del sistema di raccomandazione. È stata adottata una programmazione ad oggetti. In questa sede non viene riportato il codice però esso può essere reperito al seguente link: [https://github.com/SimoneAvellino/FacultiesRecommender](https://github.com/SimoneAvellino/FacultiesRecommender) dentro la cartella /src.

![./img/uml.jpeg](./img/uml.jpeg)

# Utilizzo

Per il corretto funzionamento deve essere presente dentro la cartella "data" il file "glove.6B.50d.w2vformat.txt", scaricabile al seguente link -> [glove](https://nlp.stanford.edu/data/glove.6B.zip). 
SE NON è PRESENTE SI DEVE AGGIUNGERE COME PRIMA RIGA LA STRINGA "400000 50" ALTRIMENTI LA LIBRERIA GLOVE NON FUNZIONA.

![./img/glove.png](./img/glove.png)

Assicurasi che il database sia avviato e che le credenziali (username, password, host) all'interno del file 'src/recommender/university.py' siano corrette.

In [ ]:
from src.recommender.recommender import Recommender
from src.recommender.query import UserQuery

recommender = Recommender()

while True:

    query = UserQuery([], "")
    query.ask_quetions_on_jupiter()
    query.show_interests()

    results = recommender.recommend(query)
    print("\nTop 5 faculties: \n")
    i = 1
    for faculty, score in results[:5]:
        print(f"{i})", faculty.get_name())
        i += 1
    
    input("")


faculties_recommender root  localhost
Connected to the MySQL database

Answer the following questions:
Quale tipologia d'istituto superiore hai frequentato?
1 ) Artistico
2 ) Musicale
3 ) Classico
4 ) Linguistico
5 ) Scienze umane
6 ) Economico
7 ) Tecnologico
8 ) Scientifico
9 ) Professionale


##  Valutazione

Ecco un'analisi sull'utilizzo del sistema con studenti universitari e un confronto tra i risultati previsti e la facoltà scelta dagli studenti. Sarà data l'opportunità di provare il sistema agli studenti che hanno compilato il form google, a coloro che non l'hanno fatto e a ChatGPT.

### Prova del sistema tramite ChatGPT

Viene dato in input a ChatGPT la seguente stringa:

"Immedesimati in uno studente delle scuole superiori che vuole scegliere una facoltà però è indeciso. Ti viene chiesto di scrivere un testo dove parli di ciò che ti piace e non ti piace (rispondi in inglese)."

Successivamente viene chiesto di rispondere alle domande del form

"Sulla base di quello che hai scritto rispondi ad ognuna di queste 8 domande. Per ogni domanda devi dare una sola risposta.

Quale tipologia d'istituto superiore hai frequentato?
- 1 ) Artistico
- 2 ) Musicale
- 3 ) Classico
- 4 ) Linguistico
- 5 ) Scienze umane
- 6 ) Economico
- 7 ) Tecnologico
- 8 ) Scientifico
- 9 ) Professionale

Preferisco studiare materie:
- 1 ) Scientifiche
- 2 ) Umanistiche

Preferisco:
- 1 ) Pratica
- 2 ) Teoria

Studio perché voglio aiutare le persone dal punto di vista: (ciò che preferisci di più)
- 1 ) artistico (es. Stilista, Designer, ...)
- 2 ) culinario (es. Cuoco, ...)
- 3 ) turistico (es. Guida Turistica, ...)
- 4 ) sociale (es. Assistente Sociale, ...)
- 5 ) educativo (es. Insegnante, ...)
- 6 ) civile (es. Avvocato, Politico, ...)
- 7 ) ambientale (es. Agricoltore, Attivista, ...)
- 8 ) economico (es. Impiegato in banca, Economista, ...)
- 9 ) tecnologico (es. Sviluppatore, Progettista, ...)
- 10 ) sanitario (es. Dottore, Infermiere, Psicologo, ...)
- 11 ) della ricerca (es. Fisico, Matematico, Storico, ...)

Mi soddisfa più: (ciò che preferisci di più)
- 1 ) creare
- 2 ) comunicare
- 3 ) progettare
- 4 ) comprendere
- 5 ) analizzare
- 6 ) ricercare
- 7 ) aiutare

Quando studio, mi piace avere più risposte per la stessa domanda:
- 1 ) No
- 2 ) Sì

Fra questi obiettivi mi rispecchio più in: (ciò che preferisci di più)
- 1 ) espandere le mie conoscenze e competenze
- 2 ) fare carriera
- 3 ) esplorare nuove culture
- 4 ) esprimere la mia creatività ed originalità
- 5 ) contribuire al benessere degli altri
- 6 ) fare una differenza nel mondo
- 7 ) promuovere la giustizia
- 8 ) promuovere l'innovazione ed il cambiamento

In quale ambiente di lavoro ti senti più a tuo agio?
- 1 ) Un ambiente creativo e innovativo
- 2 ) Un ambiente dinamico e stimolante
- 3 ) Un ambiente collaborativo e team-oriented
- 4 ) Un ambiente multiculturale e diversificato
- 5 ) Un ambiente autonomo e indipendente
- 6 ) Un ambiente strutturato e ben organizzato
"


Le risposte verranno date in input al sistema di raccomandazione e l'output dato a ChapGPT chiedendo se è soddisfatto della raccomandazione.

##### Prova 1

Interest text:

 Choosing a college major is a bit like navigating through a maze where every path seems to lead to something intriguing. As a high school student on the verge of making this colossal decision, I find myself torn between what I love and what I'm unsure about.  What I love? Well, I've always had a passion for unraveling the mysteries of the natural world. Science, with its boundless questions and endless discoveries, beckons to me like a siren's call. Whether it's exploring the intricacies of the human body in biology or delving into the cosmos in astronomy, the prospect of contributing to our understanding of the universe fills me with excitement.  However, amidst this fervor for science lies a lingering uncertainty. While I'm drawn to the allure of research and exploration, I can't shake the nagging doubt about the practicality of pursuing such a path. Will I be able to find stable employment? Will the job market be saturated with qualified candidates? These questions cast a shadow over my enthusiasm and make me second-guess my choices.  On the flip side, there are fields that I find less appealing. Finance, for instance, with its focus on numbers and profit margins, leaves me feeling cold. While I understand its importance in the functioning of society, the thought of spending my days crunching numbers and analyzing market trends fails to ignite any passion within me.  Similarly, the prospect of a career in law leaves me feeling ambivalent. While I admire the pursuit of justice and the ability to advocate for others, the rigid structure of legal proceedings and the often adversarial nature of the profession give me pause. I fear becoming trapped in a job that leaves me feeling unfulfilled and disconnected from my true passions.  In the end, my journey towards choosing a college major is a complex one, filled with excitement, uncertainty, and self-reflection. While I may not have all the answers right now, I trust that with careful consideration and introspection, I will eventually find my way to a path that not only aligns with my interests but also allows me to make a meaningful contribution to the world. 


Answers to the questions:

 1) Quale tipologia d'istituto superiore hai frequentato? : Scientifico
 2) Preferisco studiare materie: : Scientifiche
 3) Preferisco: : Teoria
 4) Studio perché voglio aiutare le persone dal punto di vista: (ciò che preferisci di più) : sanitario (es. Dottore, Infermiere, Psicologo, ...)
 5) Mi soddisfa più: (ciò che preferisci di più) : comprendere
 6) Quando studio, mi piace avere più risposte per la stessa domanda: : Sì
 7) Fra questi obiettivi mi rispecchio più in: (ciò che preferisci di più) : fare una differenza nel mondo
 8) In quale ambiente di lavoro ti senti più a tuo agio? : Un ambiente strutturato e ben organizzato

Top 5 faculties: 

1) Scienze chimiche
2) Informatica
3) Scienze e tecnologie farmaceutiche
4) Ingegneria gestionale
5) Scienze biologiche

![./img/valutazione_chatgpt1.png](./img/valutazione_chatgpt1.png)

##### Prova 2

Interest text:

 As a high school student trying to choose a college major, I find myself torn between various options, each seeming appealing in its own way. Here's a reflection on what I like and dislike:  What I like:  Creativity: I love the idea of pursuing a field that allows me to express my creativity. Whether it's through writing, designing, or problem-solving, I want a major that encourages thinking outside the box. Impact: Making a difference in the world is important to me. I'm drawn to majors where I can see tangible results and contribute positively to society, whether it's through environmental sustainability, social justice, or healthcare. Flexibility: I appreciate majors that offer versatility in career paths. I want to keep my options open and have the freedom to explore different industries and roles throughout my career. What I don't like:  Monotony: The thought of being stuck in a repetitive, mundane job terrifies me. I want a major that keeps me engaged and challenged, with opportunities for growth and change. Limited opportunities: I'm wary of choosing a major that could potentially lead to a narrow range of job prospects. I want something that opens doors rather than closes them. Lack of passion: Ultimately, I don't want to end up in a career that I'm not passionate about. I want to wake up excited to go to work every day, knowing that I'm doing something meaningful and fulfilling. In weighing these likes and dislikes, I hope to find a major that not only aligns with my interests and strengths but also offers the potential for a rewarding and fulfilling career path. 


Answers to the questions:

 1) Quale tipologia d'istituto superiore hai frequentato? : Scientifico
 2) Preferisco studiare materie: : Scientifiche
 3) Preferisco: : Teoria
 4) Studio perché voglio aiutare le persone dal punto di vista: (ciò che preferisci di più) : sociale (es. Assistente Sociale, ...)
 5) Mi soddisfa più: (ciò che preferisci di più) : creare
 6) Quando studio, mi piace avere più risposte per la stessa domanda: : Sì
 7) Fra questi obiettivi mi rispecchio più in: (ciò che preferisci di più) : fare una differenza nel mondo
 8) In quale ambiente di lavoro ti senti più a tuo agio? : Un ambiente collaborativo e team-oriented

Top 5 faculties: 

1) Scienze del turismo
2) Classe delle lauree magistrali in Giurisprudenza
3) Lettere
4) Scienze dell'economia e della gestione aziendale
5) Biologia

![./img/valutazione_chatgpt2.png](./img/valutazione_chatgpt2.png)

##### Prova 3

Interest text:

 Choosing a college major can be a daunting task, especially when you're drawn to the humanities but unsure of which path to take. As a high school student grappling with this decision, I find myself torn between my passions and practicality.  What I love about the humanities is their ability to unravel the complexities of the human experience. Literature, history, philosophy—they all offer profound insights into our culture, values, and identities. I'm fascinated by the power of words to shape societies and individuals, and I thrive on exploring different perspectives and narratives.  One aspect I particularly enjoy is the opportunity for critical thinking and analysis. Delving into a piece of literature or dissecting a historical event allows me to exercise my intellect and engage with challenging ideas. There's a sense of discovery and intellectual fulfillment that comes with unraveling the layers of meaning within a text or historical context.  Moreover, the humanities foster creativity and expression. Whether through writing essays, crafting arguments, or interpreting art, I relish the chance to express myself and contribute my unique voice to the conversation. There's something deeply satisfying about the creative process and the sense of accomplishment that comes with producing something meaningful.  However, despite my passion for the humanities, there are some concerns lingering in the back of my mind. One of the main challenges I foresee is the perceived lack of practicality in pursuing a degree in this field. In a world increasingly focused on STEM disciplines and job market demands, I worry about the viability of a career path rooted in the humanities.  There's also the issue of societal expectations and pressure to choose a "practical" major with clear job prospects. While I'm passionate about the humanities, I'm aware of the prevailing narrative that suggests pursuing such interests may lead to limited career opportunities or financial stability.  Nevertheless, I believe that following my passion is crucial for personal fulfillment and long-term success. Despite the challenges and uncertainties, I'm determined to pursue a path that aligns with my interests and values. Whether it leads to a career in academia, journalism, or another field, I'm confident that studying the humanities will equip me with valuable skills and perspectives that are relevant across diverse industries.  In conclusion, while I may face doubts and uncertainties along the way, my love for the humanities ultimately outweighs any concerns. I'm excited to embark on this journey of self-discovery and intellectual growth, confident that it will lead to a fulfilling and meaningful future. 


Answers to the questions:

 1) Quale tipologia d'istituto superiore hai frequentato? : Scienze umane
 2) Preferisco studiare materie: : Umanistiche
 3) Preferisco: : Teoria
 4) Studio perché voglio aiutare le persone dal punto di vista: (ciò che preferisci di più) : sociale (es. Assistente Sociale, ...)
 5) Mi soddisfa più: (ciò che preferisci di più) : comprendere
 6) Quando studio, mi piace avere più risposte per la stessa domanda: : Sì
 7) Fra questi obiettivi mi rispecchio più in: (ciò che preferisci di più) : esplorare nuove culture
 8) In quale ambiente di lavoro ti senti più a tuo agio? : Un ambiente collaborativo e team-oriented

Top 5 faculties: 

1) Scienze della comunicazione
2) Beni culturali
3) Informatica
4) Lingue e culture moderne
5) Ingegneria civile e ambientale

![./img/valutazione_chatgpt2.png](./img/valutazione_chatgpt3.png)

### Prova del sistema con persone che hanno compilato il form

È stato chiesto di usare il sistema di raccomandazione a persone che hanno compilato il form. Per ogni utilizzatore ci aspettiamo che fra le top 5 facoltà consigliate ci sia la facoltà che frequenta l'utente.

##### Prova 1

Lo studente frequenta informatica

Interest text:

I love using computers and playing games, i'm very creative so i would love creating my own videogame. I like math but i'm not very good at it. I hate latin and ancient languages, i don't really enjoy humanistic subjects.


Answers to the questions:

 1) Quale tipologia d'istituto superiore hai frequentato? : Scientifico
 2) Preferisco studiare materie: : Scientifiche
 3) Preferisco: : Pratica
 4) Studio perché voglio aiutare le persone dal punto di vista: (ciò che preferisci di più) : tecnologico (es. Sviluppatore, Progettista, ...)
 5) Mi soddisfa più: (ciò che preferisci di più) : creare
 6) Quando studio, mi piace avere più risposte per la stessa domanda: : Sì
 7) Fra questi obiettivi mi rispecchio più in: (ciò che preferisci di più) : esprimere la mia creatività ed originalità
 8) In quale ambiente di lavoro ti senti più a tuo agio? : Un ambiente dinamico e stimolante

Top 5 faculties: 

1) Informatica
2) Fisica
3) Ingegneria informatica
4) Ingegneria industriale
5) Scienze della nutrizione umana

Il sistema ha funzionato correttamente e lo stuende è stato soddisfatto delle altre scelte (un po' meno di "Scienze della nutrizione umana")

##### Prova 2

Lo studentessa frequenta informatica

Interest text:

I need some advice on choosing my university path. My interests vary from the modern-technological aspect to old-fashioned philosophy. I like listening to music and surfing the internet to discover new worlds and keep up with what's going on every day. I like to eat sushi in general and I detest guided excursions. Otherwise I am a person who is currently mainly dedicated to study and I would say that this is quite rewarding.


Answers to the questions:

 1) Quale tipologia d'istituto superiore hai frequentato? : Tecnologico
 2) Preferisco studiare materie: : Scientifiche
 3) Preferisco: : Pratica
 4) Studio perché voglio aiutare le persone dal punto di vista: (ciò che preferisci di più) : economico (es. Impiegato in banca, Economista, ...)
 5) Mi soddisfa più: (ciò che preferisci di più) : analizzare
 6) Quando studio, mi piace avere più risposte per la stessa domanda: : No
 7) Fra questi obiettivi mi rispecchio più in: (ciò che preferisci di più) : espandere le mie conoscenze e competenze
 8) In quale ambiente di lavoro ti senti più a tuo agio? : Un ambiente dinamico e stimolante

Top 5 faculties: 

1) Medicina e Chirurgia 
2) Informatica 
3) Ingegneria informatica 
4) Scienze della comunicazione
5) Scienze storiche

Informatica non è stata consigliata come prima scelta però è sempre sul podio. La ragazza ha affermato che ancor prima di scegliere informatica aveva valutato anche Medicina.

##### Prova 3

Lo studentessa frequenta Lettere

COMPILARE

### Prova del sistema con studenti delle scuole superiori

##### Prova 1

Interest text:

 I'm seventeen years old, I study computer science but I really like comunication, I personally think that my best quality is to convince people do something. I like changing enviroment and meeting new people, I'm really curious so I do a lot of question when I meet them. I don't know what I would like to do in the future but I general i like the financial sector 


Answers to the questions:

 1) Quale tipologia d'istituto superiore hai frequentato? : Tecnologico
 2) Preferisco studiare materie: : Umanistiche
 3) Preferisco: : Teoria
 4) Studio perché voglio aiutare le persone dal punto di vista: (ciò che preferisci di più) : economico (es. Impiegato in banca, Economista, ...)
 5) Mi soddisfa più: (ciò che preferisci di più) : comunicare
 6) Quando studio, mi piace avere più risposte per la stessa domanda: : Sì
 7) Fra questi obiettivi mi rispecchio più in: (ciò che preferisci di più) : fare carriera
 8) In quale ambiente di lavoro ti senti più a tuo agio? : Un ambiente dinamico e stimolante

Top 5 faculties: 

1) Lingue e letterature moderne europee e americane
2) Scienze biologiche
3) Informatica
4) Scienze chimiche
5) Matematica

Lo studente non è soddisfatto delle raccomandazioni da parte del sistema. L'utente si aspettava in output tutto tranne materie scientifiche.

##### Prova 2

DA FINIRE

# Conclusione

Il sistema ha ottenuto risultati soddisfacenti nel confronto con gli studenti già iscritti all'università, ma ha ricevuto recensioni deludenti da parte degli studenti delle scuole superiori. Questo fenomeno sembra essere correlato a diversi fattori:

- **Mancanza di informazioni sugli studenti iscritti alle facoltà:** Potrebbe essere utile avviare una campagna per diffondere il questionario e incoraggiare la partecipazione di un numero maggiore di studenti, al fine di rappresentare una gamma più ampia di facoltà.

- **Scarsità di dettagli nelle descrizioni delle facoltà utilizzate nell'analisi TF-IDF:** È necessario arricchire le descrizioni delle facoltà per garantire una ricerca più accurata e pertinente in base agli interessi degli utenti.

- **Miglioramento dell'analisi del testo:** L'analisi del testo deve essere ottimizzata per evitare la classificazione errata di parole come interessi positivi o negativi, specialmente quando il loro significato reale è contrario alla valutazione assegnata.

Si consiglia quindi di concentrarsi su questi punti chiave per migliorare l'esperienza complessiva degli studenti e rendere il sistema di orientamento universitario più efficace ed inclusivo.

Un'ulteriore miglioramento potrebbe consistere nell'introdurre domande aggiuntive quando il sistema rileva una mancanza di informazioni sufficienti, come un testo sugli interessi dell'utente troppo breve o ambiguo. Questo approccio mira a guidare l'utente verso una selezione più precisa attraverso un'interazione più approfondita. Le domande supplementari possono aiutare a raccogliere dettagli specifici e rilevanti per offrire raccomandazioni universitarie più accurate e personalizzate.

Un'altra proposta emersa da diversi studenti che hanno compilato il modulo è l'introduzione di risposte multiple alle domande. Questo miglioramento consentirebbe agli utenti di fornire risposte più dettagliate e accurate, tenendo conto della complessità e della diversità delle loro preferenze e interessi. Implementare risposte multiple potrebbe aumentare l'efficacia del sistema nell'offrire raccomandazioni universitarie più mirate e pertinenti.

